<a href="https://colab.research.google.com/github/aliash98/BPM_CausalReasoning/blob/main/Idenitification%26Estimation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ylearn

In [ ]:
!pip install dowhy

In [ ]:
!pip install networkx==2.8.5

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip list

In [ ]:
import numpy as np
import pandas as pd
from ylearn.causal_model.graph import CausalGraph
from ylearn.causal_model import CausalModel
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression

In [ ]:
# Synthesis of Data based on the causal graph Figure 1 (a) --- Assuming normal distribution for both

# V3 is the risk taking indicator of a client
# V1 is the amount a client has asked for a loan (in thousand dollars)
# V2 is the client's credit score
# D is the decision made 

In [ ]:
# Synthesis based on the causal graph Figure 1 (c) --- 

# define the conditional probabilities
P_v3 = 0.5
P_v2_given_v3 = np.array([0.4, 0.5])
P_v1_given_v3_v2 = np.array([[0.6, 0.4], [0.4, 0.3]])
P_d_given_v2_v1 = np.array([[0.4, 0.3], [0.7, 0.35]])

# generate 1000 samples
n_samples = 100000
samples = np.zeros((n_samples, 4))
for i in range(n_samples):
    # sample v3
    v3 = np.random.binomial(1, P_v3)
    # sample v2 given v3
    p_v2_given_v3 = P_v2_given_v3[v3]
    v2 = np.random.binomial(1, p_v2_given_v3)
    # sample v1 given v3 and v2
    p_v1_given_v3_v2 = P_v1_given_v3_v2[v3][v2]
    v1 = np.random.binomial(1, p_v1_given_v3_v2)
    # sample d given v2 and v1
    p_d_given_v2_v1 = P_d_given_v2_v1[v2][v1]
    d = np.random.binomial(1, p_d_given_v2_v1)
    # store the sample
    samples[i] = [v3, v2, v1, d]

In [ ]:
# Converting array to dataframe
samples_df = pd.DataFrame({'V3': samples[:, 0], 'V2': samples[:, 1], 'V1':samples[:, 2], 'D':samples[:, 3]})

samples_df

In [ ]:
# Testing the accuracy of sampling

# k = samples_df[(samples_df['V3'] == 0) & (samples_df['V2'] == 1)]

samples_df.groupby(['V2', 'V1'])['D'].mean()
# Works perfectly

V2   V1 
0.0  0.0    0.402981
     1.0    0.296415
1.0  0.0    0.699585
     1.0    0.350179
Name: D, dtype: float64

In [ ]:
# Constructing the Motivating example graph

# Here we define observed variables
causation = {
        'V2': [],
        'V1': ['V2'],
        'D': ['V1', 'V2'],
}

# Here we define that V1 and V2 are connected with a hidden confounder (V3)
arcs = [('V1', 'V2')]

# cg = CausalGraph(causation, latent_confounding_arcs=arcs)
cg = CausalGraph(causation)
cm = CausalModel(cg)

In [ ]:
# P(D| do(V1)) (which should be NOT be identifiable) ---- Throws an error saying it is not identifiable

p = cm.id({'D'}, {'V1'})
p.show_latex_expression()

<IPython.core.display.Latex object>

In [ ]:
# Let's compare it to non-model calculation

# do(V1 = 1)

# P(V2 = 1) = 0.45
# P(V2 = 0) = 0.55
# P(D = 1 | V1 = 1, V2 = 1) = 0.35
# P(D = 1 | V1 = 1, V2 = 0) = 0.3


# Statistical estimand: 
# P(D = 1 | V1 = 1, V2 = 0) * P(V2 = 0) + P(D = 1 | V1 = 1, V2 = 1) * P(V2 = 1)
# = 0.3 * 0.55 + 0.35 * 0.45 = 0.3225

In [ ]:
# do(V1 = 0)

# P(V2 = 1) = 0.45
# P(V2 = 0) = 0.55
# P(D = 1 | V1 = 0, V2 = 1) = 0.7
# P(D = 1 | V1 = 0, V2 = 0) = 0.4


# Statistical estimand: 
# P(D = 1 | V1 = 0, V2 = 0) * P(V2 = 0) + P(D = 1 | V1 = 0, V2 = 1) * P(V2 = 1)
# = 0.4 * 0.55 + 0.7 * 0.45 = 0.535

In [ ]:
# A simple logistic regression

X = samples_df[['V1', 'V2']]    # Features
y = samples_df.D                # Target variable

# For now we don't split train / test
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=16)

In [ ]:
logreg = LogisticRegression(random_state=16)
logreg.fit(X, y)

LogisticRegression(random_state=16)

In [ ]:
# do(V1 = 1)
X_pred = X.copy()
X_pred['V1'] = 1 
y_pred_1 = logreg.predict(X_pred)
y_pred_1.mean()

0.0

In [ ]:
# do(V1 = 0)
X_pred = X.copy()
X_pred['V1'] = 0 
y_pred_0 = logreg.predict(X_pred)
y_pred_0.mean()

0.44978

In [ ]:
linreg = LinearRegression()
linreg.fit(X, y)

LinearRegression()

In [ ]:
# do(V1 = 1)
X_pred = X.copy()
X_pred['V1'] = 1 
y_pred_1 = linreg.predict(X_pred)
y_pred_1.mean()

0.3335987649038346

In [ ]:
# do(V1 = 0)
X_pred = X.copy()
X_pred['V1'] = 0 
y_pred_0 = linreg.predict(X_pred)
y_pred_0.mean()

0.5435394373313088

In [ ]:
# Print ATE
print(y_pred_1.mean() - y_pred_0.mean())

-0.20994067242747422


In [ ]:
import networkx as nx
import dowhy.gcm as gcm

# we got the data

causal_model = gcm.ProbabilisticCausalModel(nx.DiGraph([('V3', 'V2'), ('V3', 'V1'), ('V2', 'V1'), ('V1', 'D'), ('V2', 'D')]))
gcm.auto.assign_causal_mechanisms(causal_model, samples_df)
gcm.fit(causal_model, samples_df)

Fitting causal mechanism of node D: 100%|██████████| 4/4 [00:08<00:00,  2.23s/it]


In [ ]:
gcm.average_causal_effect(causal_model,
                         'D',
                         interventions_alternative={'V1': lambda x: 1},
                         interventions_reference={'V1': lambda x: 0},
                         num_samples_to_draw=1000)

In [ ]:
gcm.average_causal_effect(causal_model,
                         'D',
                         interventions_alternative={'V1': lambda x: 1},
                         interventions_reference={'V1': lambda x: 0},
                         observed_data=samples_df)

In [ ]:
# Average Treatment Effect (ATE) E(D|do(V1=1)) - E(D|do(V1=0)) = E[D(1) - D(0)] = P(D=1|do(V1=1)) - P(D=1|do(V1=0))

# Without a model:               0.3225 - 0.535 = -0.2125  (just probability calculation)
# Logistic Regression:              0 - 0.44978 = -0.45 
# Linear Regression:              0.33 - 0.5435 = -0.20994 
# DoWhy package estimator:                        −0.2313 (with sampled data)
# DoWhy package estimator:                        −0.2148 (with original data)